In [ ]:
%%bash
source $VIRTUAL_ENV_DIR/python3/bin/activate

install_package_python3.sh add dsw_qr==0.1.13

$VIRTUAL_ENV_DIR/python3/bin/python -m pip install galileo
$VIRTUAL_ENV_DIR/python3/bin/python -m pip install galileo-py
$VIRTUAL_ENV_DIR/python3/bin/python -m pip install tchannel

In [ ]:
import os

import pandas as pd
from dsw_qr import dsw_qr
from queryrunner_client import Client
qr = Client(user_email='thai@uber.com')

In [ ]:
def prepare_query(city_id, date, cal_type):
    QUERY = f"""
    SET session hash_partition_count=64;

    select DISTINCT
        msg.job_upfront_fare as upfront_fare_local,
        trip.client_upfront_fare_local,
        trip.client_upfront_fare_usd as upfront_fare_usd,
        vc.gross_bookings_usd as gb_usd
    from   
        rawdata.kafka_hp_multileg_mgv_log_nodedup mgv
    join
        dwh.fact_trip trip
    on
        mgv.datestr = trip.datestr
        and msg.city_id = trip.city_id
        and msg.job_uuid = trip.uuid
    join
        secure_finance.fds_rides_vc vc
    on
        mgv.datestr = vc.operational_date
        and msg.city_id = vc.city_id
        and msg.job_uuid = vc.job_uuid
    where
        mgv.datestr = '{date}'
        and msg.city_id = {city_id}
        and msg.tenancy = 'uber/production'
        and msg.flow_type = 'solo_batch'
        and msg.job_uuid <> msg.client_uuid
        and msg.calculator_type = '{cal_type}'
        and lower(trip.global_product_name) = 'uberx'
        and trip.status = 'completed'
    """

    return QUERY

In [ ]:
city_list = [797, 1269, 38, 34, 36]
date_list = ['2022-10-01', '2022-10-01', '2022-10-15', '2022-10-15', '2022-10-15']
cal_type_list = ['machine_learning_of', 'machine_learning_of', 'machine_learning_emea_of', 'machine_learning_emea_of', 'machine_learning_emea_of']

In [ ]:
for i in range(5):
    city_id = city_list[i]
    date = date_list[i]
    cal_type = cal_type_list[i]
    QUERY = prepare_query(city_id, date, cal_type)
    cursor = qr.execute("presto-secure", QUERY)
    result = cursor.fetchall()
    pd.DataFrame(result).to_csv(f"mlof_gb_plot_fare_city_{city_id}.csv", index=False)